<a href="https://colab.research.google.com/github/shabathink/WholeLearnerBot/blob/main/WholeLearnerBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pickle

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/codeNLP/data/learner_inputs.csv')
df.head()

,input_text,emotion,motivation,expected_bloom
0,I feel stuck,frustration,low,apply
1,I'm improving in this topic,joy,high,evaluate
2,I'm confused again,frustration,low,understand
3,This is going great!,joy,high,create
4,I'm totally lost,sadness,low,remember


In [3]:
# Train emotion classifier
import os # Import the os module

vec = TfidfVectorizer()
X = vec.fit_transform(df['input_text'])
y = df['emotion']
model = LogisticRegression().fit(X, y)

# Create the 'models' directory if it doesn't exist
if not os.path.exists('models'):
    os.makedirs('models')

with open('models/emotion_classifier.pkl', 'wb') as f:
    pickle.dump((model, vec), f)
print("Classifier trained and saved.")

Classifier trained and saved.


In [4]:
# Train emotion classifier
vec = TfidfVectorizer()
X = vec.fit_transform(df['input_text'])
y = df['emotion']
model = LogisticRegression().fit(X, y)
with open('models/emotion_classifier.pkl', 'wb') as f:
    pickle.dump((model, vec), f)
print("Classifier trained and saved.")

Classifier trained and saved.


In [5]:
# Predict and evaluate
df['predicted'] = model.predict(X)
df['correct'] = df['predicted'] == df['emotion']
accuracy = df['correct'].mean()
print(f"Prediction accuracy: {accuracy:.2f}")

Prediction accuracy: 1.00


In [6]:
from sentence_transformers import SentenceTransformer, util

# Load learning objectives
sef_model = SentenceTransformer('all-MiniLM-L6-v2')
objectives = pd.read_csv('/content/drive/MyDrive/codeNLP/data/learning_objectives.csv')

def compute_sef(response, target):
    emb1 = sef_model.encode(response, convert_to_tensor=True)
    emb2 = sef_model.encode(target, convert_to_tensor=True)
    return float(util.cos_sim(emb1, emb2))

# Example SEF calculation
response = "Let's try solving an example together."
target = objectives.loc[1, 'objective']
print(f"SEF score: {compute_sef(response, target):.2f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SEF score: 0.34


In [13]:
import pandas as pd            # make sure pandas is imported

log_df = pd.DataFrame({
    'user_id':   [1, 2],
    'input_text': ['I feel stuck', 'This is going great!'],
    'emotion':   ['frustration', 'joy'],
    'tier':      [1, 3],
    'response':  ["Let's try an example.",  # ← fixed quotes
                  'Try designing your own version!'],
    'sef_score': [0.89, 0.91]
})

# Make sure the folder exists before writing
# (e.g. Path('results').mkdir(exist_ok=True))
log_df.to_csv('/content/drive/MyDrive/codeNLP/results/logs.csv', index=False)
log_df


,user_id,input_text,emotion,tier,response,sef_score
0,1,I feel stuck,frustration,1,Let's try an example.,0.89
1,2,This is going great!,joy,3,Try designing your own version!,0.91
